# Hirarhical LDA

Here instead of LDA I used hierarchical dirichlet process. I was surprised by two things. First, how much quicker it was then LDA and second, how less interpretable results did I get. (But if I learned anything during this project it is, that my knowledge of cognitive sciences is limited).

In [1]:
import os
import nltk
import string
import gensim
import pandas
import collections
from constants import all_topics_names, folder_meicogsci, folder_meicogsci_2019, folder_models, folder_vis

In [2]:
# get list of abstracts
all_files = os.listdir(folder_meicogsci)
all_files_2019 = os.listdir(folder_meicogsci_2019)

In [3]:
#get helping functions
stop_words = set(nltk.corpus.stopwords.words("english"))
lem = nltk.stem.WordNetLemmatizer()
stem = nltk.stem.porter.PorterStemmer()

In [4]:
#preprocess the text
text_words_all_words_noprocess = collections.defaultdict(dict)
for folder, filenames in [[folder_meicogsci, all_files], [folder_meicogsci_2019, all_files_2019]]:
    for filename in filenames:
        text_name = filename.split("_")[-1].replace(".txt", "")
        with open(os.path.join(folder, filename)) as f:
            data_file_raw = f.readlines()
        # the following part is here to eliminate the rows, that are just citations, since they are providing noise
        data_file = data_file_raw[:-7][:]
        for row in data_file_raw[-7:][:]:
            row = row.strip()
            if len(row) > 2:
                if not row[0] == "[" and not row[2] == "]":
                    data_file.append(row)
        # the following part is there to eliminate the rown, that are just thanks
        data_file = [row for row in data_file if "special thanks" not in row.lower()]
        data_file = [row for row in data_file if "acknowledgements" not in row.lower()]
        data_file = [row for row in data_file if "I would like to thank" not in row.lower()]
        data_file = [row for row in data_file if "we thank" not in row.lower()]
        data_file = [row for row in data_file if "special thanks" not in row.lower()]
        data_file = [row for row in data_file if "sincere thanks" not in row.lower()]
        data_file = [row for row in data_file if "many thanks" not in row.lower()]
        # the following part is there to eliminate the abstract and reference header
        data_file = [row for row in data_file if "abstract" not in row.lower() or len(row) > 25]
        data_file = [row for row in data_file if "reference" not in row.lower() or len(row) > 25]
        data_file = [row for row in data_file if "university press" not in row.lower()]
        # the the text is being put in the format, that makes it more informative to compare
        data_file = " ".join(data_file)
        data_file = data_file.replace("... title:", "")
        data_file = data_file.lower()
        data_file = data_file.replace("-", " ").replace("/", " ").replace("–", " ") 
        # here, the test is being split into words and compared based on that
        data_words = nltk.tokenize.word_tokenize(data_file)
        data_words = [w.strip() for w in data_words if w.strip()]
        # this is prefiltering based on 1-grams
        data_words_filtered_nltk = nltk.pos_tag(data_words)
        data_words_filtered = [w for w, pos in data_words_filtered_nltk if pos[:2] in {"JJ", "NN"}]
        data_words_filtered = [w for w in data_words_filtered if w not in stop_words and w not in string.punctuation and w not in "`’”“"]
        data_words_filtered = [w for w in data_words_filtered if len(w) > 2]
        data_words_filtered = [lem.lemmatize(w) for w in data_words_filtered]
        text_words_all_words_noprocess[text_name][1] = data_words_filtered
        # this is prefiltering based on 2-grams
        data_bigrams = nltk.bigrams(data_words)
        data_bigrams = [(w1, w2) for w1, w2 in data_bigrams if w1 not in string.punctuation and w2 not in string.punctuation]
        data_bigrams = [(w1, w2) for w1, w2 in data_bigrams if w1 not in "`’”“" and w2 not in "`’”“"]
        data_bigrams = [(w1, w2) for w1, w2 in data_bigrams if w1 not in stop_words and w2 not in stop_words]
        data_bigrams = [(w1, w2) for w1, w2 in data_bigrams if nltk.pos_tag(w1)[0][1][:2] in {"JJ", "NN"} and nltk.pos_tag(w2)[0][1][:2] in {"JJ", "NN"}]
        data_bigrams = [(lem.lemmatize(w1), lem.lemmatize(w2)) for w1, w2 in data_bigrams]
        text_words_all_words_noprocess[text_name][2] = data_bigrams
        # this is prefiltering based on 3-grams
        data_trigrams = nltk.ngrams(data_words, 3)
        data_trigrams = [(w1, w2, w3) for w1, w2, w3 in data_trigrams if w1 not in stop_words and w2 not in stop_words and w3 not in stop_words]
        data_trigrams = [(w1, w2, w3) for w1, w2, w3 in data_trigrams if w1 not in string.punctuation and  w2 not in string.punctuation and w3 not in string.punctuation]
        data_trigrams = [(w1, w2, w3) for w1, w2, w3 in data_trigrams if w1 not in "`’”“" and w2 not in "`’”“" and w3 not in "`’”“"]
        data_trigrams = [(w1, w2, w3) for w1, w2, w3 in data_trigrams if nltk.pos_tag([w1])[0][1][:2] in {"JJ", "NN"} and nltk.pos_tag([w2])[0][1][:2] in {"JJ", "NN"} and nltk.pos_tag([w3])[0][1][:2] in {"JJ", "NN"}]
        data_trigrams = [(lem.lemmatize(w1), lem.lemmatize(w2), lem.lemmatize(w3)) for w1, w2, w3 in data_trigrams]
        text_words_all_words_noprocess[text_name][3] = data_trigrams

In [5]:
#check how many files were preprocessed
number_of_files = len(text_words_all_words_noprocess)
number_of_files

848

In [6]:
#preprocess all one-grams
all_one_grams = []
for text_id, info in text_words_all_words_noprocess.items():
    all_one_grams += set(info[1])

In [7]:
all_one_grams = collections.Counter(all_one_grams)

In [8]:
keep_one_grams = set([w for w, count in all_one_grams.items() if count >= 5])

In [9]:
#preprocess all one-grams
all_two_grams = []
for text_id, info in text_words_all_words_noprocess.items():
    all_two_grams += set(info[2])

In [10]:
all_two_grams = collections.Counter(all_two_grams)

In [11]:
keep_two_grams = set([w for w, count in all_two_grams.items() if count >= 5])

In [12]:
#preprocess all one-grams
all_three_grams = []
for text_id, info in text_words_all_words_noprocess.items():
    all_three_grams += set(info[3])

In [13]:
all_three_grams = collections.Counter(all_three_grams)

In [14]:
keep_three_grams = set([w for w, count in all_three_grams.items() if count >= 5])

In [15]:
#put all preprocessed words together
text_words_all_words_process = collections.defaultdict(dict)
for text_id, info in text_words_all_words_noprocess.items():
    text_words_all_words_process[text_id][1] = [w for w in info[1] if w in keep_one_grams]
    text_words_all_words_process[text_id][2] = [" ".join(w) for w in info[2] if w in keep_two_grams]
    text_words_all_words_process[text_id][3] = [" ".join(w) for w in info[3] if w in keep_three_grams]

In [16]:
text_words_all_words_process_list = collections.defaultdict(list)
for text_id, info in text_words_all_words_process.items():
    text_words_all_words_process_list[text_id] += info[1]
    text_words_all_words_process_list[text_id] += info[2]
    text_words_all_words_process_list[text_id] += info[3]

In [17]:
#prepare the data fo modeling
all_texts = [t for t in text_words_all_words_process_list.values()]

In [18]:
dictionary = gensim.corpora.Dictionary(all_texts)

In [19]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in all_texts]

In [20]:
#create a model
model_hdp = gensim.models.HdpModel(doc_term_matrix, id2word = dictionary)

In [21]:
#check frequency distribution of words by topic
all_terms_sum = collections.defaultdict(int)
all_topic_terms = model_hdp.show_topics(num_words=300, formatted=False)
for topic, all_terms in all_topic_terms:
    for term, value in all_terms:
        all_terms_sum[term] += value

In [22]:
# calculate absolute and relative importance of each word to a topic
current_model = model_hdp
words_represented = dict()
for topic_words in current_model.show_topics(num_words=50, formatted=False):
    current_topic = topic_words[0]
    words_represented[current_topic] = collections.defaultdict(list)
    all_words = topic_words[1]
    all_words_relative = []
    for word, weight in topic_words[1]:
        current_sum = all_terms_sum[word]
        current_relative_weight = weight/current_sum
        all_words_relative.append([current_relative_weight, weight, current_relative_weight*weight, word])
    words_represented[current_topic] = all_words_relative

In [23]:
# print 5 words with highest socre for each topic
for i in range(20):
    print(list(pandas.DataFrame(words_represented[i], columns=["Relative", "Absolute", "Score", "Word"]).sort_values(by="Score", ascending=False).head(5)["Word"]))

['language', 'research', 'participant', 'result', 'study']
['research', 'bias', 'model', 'cognitive', 'effect']
['game', 'player', 'object', 'kind', 'eye']
['emotion', 'sound', 'meditation', 'emotional', 'experience']
['necessary', 'conscious', 'science', 'scientific', 'dance']
['sound', 'duration', 'presentation', 'association', 'fluency']
['joy', 'transaction', 'agency', 'legal', 'keyboard']
['robot', 'architecture', 'michael', 'statistical method', 'relevance']
['ball', 'industry', 'dynamical', 'blood', 'pressure']
['gesture', 'visual stimulus', 'immortality', 'well established', 'expectation']
['coordinate', 'different scenario', 'theory state', 'frontal cortex', 'prediction error']
['tm', 'mep', 'rank', 'le', 'placebo']
['receptor', 'subunit', 'learning environment', 'cognitive effort', 'anova']
['spectral', 'speed', 'reliability', 'feminine', 'textual']
['dorsolateral prefrontal cortex', 'neural correlate', 'inferential', 'theory u', 'thalamus']
['sudden', 'spatial smoothing', 'c

In [24]:
# print 5 most important words in absolute sense for each topic
for i in range(20):
    print(list(pandas.DataFrame(words_represented[i], columns=["Relative", "Absolute", "Score", "Word"]).sort_values(by="Absolute", ascending=False).head(5)["Word"]))

['research', 'study', 'language', 'participant', 'result']
['research', 'cognitive', 'study', 'model', 'participant']
['language', 'data', 'study', 'game', 'result']
['emotion', 'study', 'social', 'experience', 'subject']
['cognitive', 'science', 'research', 'study', 'stimulus']
['language', 'sound', 'word', 'cognitive', 'participant']
['study', 'brain', 'result', 'method', 'research']
['robot', 'learning', 'architecture', 'theory', 'data']
['process', 'system', 'research', 'cognitive', 'social']
['task', 'motor', 'brain', 'gesture', 'cognitive']
['behaviour', 'cognitive', 'study', 'schizophrenia', 'motor']
['effect', 'tm', 'motor', 'behaviour', 'task']
['cell', 'part', 'status', 'thesis', 'strategy']
['picture', 'data', 'spectral', 'network', 'study']
['human', 'task', 'system', 'bayesian', 'dorsolateral prefrontal cortex']
['music', 'system', 'sudden', 'dominance', 'effect']
['story', 'user', 'cognitive', 'theory', 'level']
['task', 'word', 'practical', 'comparable', 'stimulus']
['sc